<a href="https://colab.research.google.com/github/mimiachiu/Deep-Learning-with-Python-Book/blob/master/Ch07_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install keras

In [ ]:
#序列式v.s函數式
from keras.models import Sequential, Model
from keras import layers, Input

#序列式
seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))
print('Sequential')
seq_model.summary()

#函數式
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
y = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(y)
fun_model = Model(input_tensor, output_tensor)
print('Functional')
fun_model.summary()

Using TensorFlow backend.


Sequential
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________
Functional
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_____________

In [ ]:
#以函數式API實作雙輸入問答模型
from keras import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None, ), dtype='int32', name='text')
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
print(embedded_text.shape)
encoded_text = layers.LSTM(32)(embedded_text)
print(encoded_text.shape)

question_input = Input(shape=(None, ), dtype='int32', name='question')
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
print(embedded_question.shape)
encoded_question = layers.LSTM(16)(embedded_question)
print(encoded_question.shape)

concatenated = layers.concatenate([encoded_question, encoded_text], axis=-1)
print(concatenated.shape)

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)
print(answer.shape)

model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

(None, None, 64)
(None, 32)
(None, None, 32)
(None, 16)
(None, 48)
(None, 500)
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
question (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
text (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 32)     320000      question[0][0]                   
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 64)     640000      text[0][0]                       
_____________

In [ ]:
#將資料以兩種方式(list、dict)傳遞到多輸入模型進行訓練
import numpy as np

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
print(text.shape)

question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))
print(question.shape)

answers = np.zeros(shape=(num_samples, answer_vocabulary_size), dtype='int32')

for answer in answers:
    answer[np.random.randint(answer_vocabulary_size)] = 1
print(answers.shape)

#方法1：使用lsit
model.fit([text, question], answers, epochs=10, batch_size=128)
#方法2：使用dict，鍵為Input層的名稱，值為Numpy資料
model.fit({'text':text, 'question':question}, answers, epochs=10, batch_size=128)

(1000, 100)
(1000, 100)
(1000, 500)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
1000/1000 [==============================] - 6s 6ms/step - loss: 6.2145 - acc: 0.0010
Epoch 2/10
1000/1000 [==============================] - 4s 4ms/step - loss: 6.1925 - acc: 0.0420
Epoch 3/10
1000/1000 [==============================] - 4s 4ms/step - loss: 6.1204 - acc: 0.0040
Epoch 4/10
1000/1000 [==============================] - 4s 4ms/step - loss: 6.0535 - acc: 0.0040
Epoch 5/10
1000/1000 [==============================] - 4s 4ms/step - loss: 5.9997 - acc: 0.0070
Epoch 6/10
1000/1000 [==============================] - 4s 4ms/step - loss: 5.9391 - acc: 0.0100
Epoch 7/10
1000/1000 [==============================] - 4s 4ms/step - loss: 5.8671 - acc: 0.0100
Epoch 8/10
1000/1000 [==============================] - 4s 4ms/step - loss: 5.7872 - acc: 0.0140
Epoch 9/10
1000/1000 [==============================] - 4s 4ms/step - loss: 5.6984 - acc: 0.0240
Epoch 10/10
1000/1000 [==============================] - 4s 4ms/step - loss: 5.6131 - acc: 0.0240
Epoch 1/10
1000/1000 [=======

In [ ]:
#以函數式API實作三個輸出結果的模型
from keras import layers, Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None, ), dtype='int32', name='posts')

embedding_posts = layers.Embedding(vocabulary_size, 256)(posts_input)
print(embedding_posts.shape)

x = layers.Conv1D(128, 5, activation='relu')(embedding_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)
print(x.shape)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])
model.summary()

(None, None, 256)
(None, 128)
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, None, 256)    12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, None, 128)    163968      embedding_6[0][0]                
__________________________________________________________________________________________________
max_pooling1d_3 (MaxPooling1D)  (None, None, 128)    0           conv1d_7[0][0]                   
______________________________________________________________

In [ ]:
#多輸出模型的編譯選項，指定多重損失函數，有兩種方式
#編譯方式1：使用損失串列(loss list)
model.compile(optimizer='rmsprop', loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'], loss_weights=[0.25, 1., 10.])
#編譯方式2：使用損失字典(loss dict)，鍵為輸出層的名稱
model.compile(optimizer='rmsprop', loss={'age':'mse', 'income':'categorical_crossentropy', 'gender':'binary_crossentropy'}, loss_weights={'age':0.25, 'income':1., 'gender':10.})

In [ ]:
#實作Inception模組
from keras import layers, Input

x = Input(batch_shape=(1000, 28, 28, 256))
print(x.shape)

branch_a = layers.Conv2D(64, 1, activation='relu', strides=2)(x)
print(branch_a.shape)

branch_b = layers.Conv2D(128, 1, activation='relu')(x)
branch_b = layers.Conv2D(128, 3, activation='relu', strides=2, padding='same')(branch_b)
print(branch_b.shape)

branch_c = layers.AveragePooling2D(3, strides=2, padding='same')(x)
branch_c = layers.Conv2D(128, 3, activation='relu', padding='same')(branch_c)
print(branch_c.shape)

branch_d = layers.Conv2D(128, 1, activation='relu')(x)
branch_d = layers.Conv2D(128, 3, activation='relu', padding='same')(branch_d)
branch_d = layers.Conv2D(128, 3, activation='relu', strides=2, padding='same')(branch_d)
print(branch_d.shape)

output = layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)
print(output.shape)

(1000, 28, 28, 256)
(1000, 14, 14, 64)
(1000, 14, 14, 128)
(1000, 14, 14, 128)
(1000, 14, 14, 128)
(1000, 14, 14, 448)


In [ ]:
#殘差連接：啟動函數輸出張量大小相同時
from keras import layers, Input

x = Input(batch_shape=(1000, 32, 32, 128))
print(x.shape)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
z = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
print(z.shape)

op = layers.add([z, x])
print(op.shape)

(1000, 32, 32, 128)
(1000, 32, 32, 128)
(1000, 32, 32, 128)


In [ ]:
#殘差連接：啟動函數輸出張量大小不同時
from keras import layers, Input

x = Input(batch_shape=(1000, 32, 32, 256))
print(x.shape)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
z = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
print(z.shape)

t = layers.MaxPooling2D(2, strides=2)(z)
print(t.shape)

residual = layers.Conv2D(128, 1, strides=2, padding='same')(x)
print(residual.shape)

op = layers.add([t, residual])
print(op.shape)

(1000, 32, 32, 256)
(1000, 32, 32, 128)
(1000, 16, 16, 128)
(1000, 16, 16, 128)
(1000, 16, 16, 128)


In [ ]:
#孿生LSTM(Siamese LSTM)/共享LSTM(Shared LSTM)
from keras import layers, Input
from keras.models import Model

lstm = layers.LSTM(32)
left_input = Input(shape=(None, 128))
print(left_input.shape)
left_output = lstm(left_input)
print(left_output.shape)

right_input = Input(shape=(None, 128))
print(right_input.shape)
right_output = lstm(right_input)
print(right_output.shape)

merged = layers.concatenate([left_output, right_output], axis=-1)
print(merged.shape)

predictions = layers.Dense(1, activation='sigmoid')(merged)
model = Model([left_input, right_input], predictions)

(None, None, 128)
(None, 32)
(None, None, 128)
(None, 32)
(None, 64)
